In [1]:
import sys
import os
root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(sys.argv[0])))))
path_data = root + "/Documents/GitHub/Kitaev_LR_QP/data/data_paper/"
path_figures = root + "/Documents/GitHub/Kitaev_LR_QP/figures/figures_paper/"
sys.path.append(root + "/Documents/GitHub/Kitaev_LR_QP/")

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec

from Topo_LR_Kitaev_AAH.Hamiltonians import *
from Topo_LR_Kitaev_AAH.Winding_num import *

# ENERGY SPECTRUM

In [ ]:
it = 12
L = 1
Fn1, Fn = Fibonacci(it)

for alpha in [0.5,5]:
    for constant in [False, True]:

        params = dict(
            alpha = alpha,
            delta = 1/2,
            t = -1,
            Fn1 = Fn1,
            Fn = Fn,
            phase = 0,
            constant= constant
        )

        energies_m = energies_r = energies_o = energies_i = []

        mu_list = np.arange(-3,3,0.2)
        m_list = np.arange(0, L)
        k_list1 = (2*m_list+1)*np.pi/(L)
        
        D = L
        delta_k = 2*np.pi/D
        k_list2 = np.arange(0, 2*np.pi, delta_k)+0.01

        for mu in mu_list:

            energies1 = energies2 = []

            H_r = H_APBC_Kitaev_LR_QP(params, mu, L, AA=True)
            evals_r, evecs = np.linalg.eigh(H_r)
            
            H_o = H_OBC_Kitaev_LR_QP(params, mu, L, AA=True)
            evals_o, evecs = np.linalg.eigh(H_o)

            if len(energies_r) == 0:
                energies_r = evals_r
            else:
                energies_r = np.vstack((energies_r, evals_r))

            if len(energies_o) == 0:
                energies_o = evals_o
            else:
                energies_o = np.vstack((energies_o, evals_o))
                
        energies_r = energies_r.T
        energies_o = energies_o.T
        
        np.save(path_data + 'energies_OBC_alpha_%.2f_constant_%s_it_%.0f_L_%.0f.npy' %(alpha, str(constant), it, L), energies_o)
        np.save(path_data + 'energies_APBC_alpha_%.2f_constant_%s_it_%.0f_L_%.0f.npy' %(alpha, str(constant), it, L), energies_r)

# REAL-SPACE WINDING NUMBER

In [ ]:
it = 12
Fn1, Fn = Fibonacci(it)
length = 1
# length 3, 5

for alpha in [0.5, 5]:
    print('alpha: ', alpha)
    
    for constant in [False, True]:
        print('constant: ', constant)

        params = dict(
            delta = 1/2,
            alpha = alpha,
            t = -1,
            Fn1 = Fn1,
            Fn = Fn,
            phase = 0,
            constant = constant
        )

        vec_mu = np.arange(-3,3,0.2)
        vec_w = np.zeros_like(vec_mu)

        for i, mu in enumerate(vec_mu):
             vec_w[i] = Real_space_Kitaev_LR_QP_wn(params, mu, length, AA=True)
                
        np.save(path_data + 'RS_winding_num_alpha_L_%.0f_%.2f_constant_%s_it_%.0f.npy' %(length, alpha, str(constant), it), vec_w)

# INFINITE SYSTEM WINDING NUMBER

In [2]:
it = 12
Fn1, Fn = Fibonacci(it)
length = 1

delta_k = 2*np.pi/30
veck = np.arange(delta_k, 2*np.pi, delta_k)
vec_mu = np.arange(-3,3,0.2)

for alpha in [0.5, 5]:
    for constant in [False, True]:
        params = dict(
            delta = 1/2,
            alpha = alpha,
            t = -1,
            Fn1 = Fn1,
            Fn = Fn,
            phase = 0,
            constant = constant
        )
        
        vec_w = Chiral_Kitaev_QP_LR_wn(params, vec_mu, veck, AA=True)
        np.save(path_data + 'C_winding_num_alpha_%.2f_constant_%s_it_%.0f.npy' %(alpha, str(constant), it), vec_w)

# CRITICAL SUPERCONDUCTING PAIRING

In [ ]:
it = 12
length = 1
Fn1, Fn = Fibonacci(it)

mu_list = np.arange(0.5,2.1,0.1)
alpha_list = np.arange(0.1,3.1,0.1)
delta_list = np.linspace(0,1,50)

for constant in [True, False]:
    
    for mu in mu_list:
        energies_array = wn_array = []
        
        for alpha in alpha_list:
            energies_r = []
            vec_w = np.zeros_like(delta_list)
            
            for i, delta in enumerate(delta_list):
                params = dict(
                    alpha = alpha,
                    delta = delta,
                    t = -1,
                    Fn1 = Fn1,
                    Fn = Fn,
                    phase = phase,
                    constant= constant
                )

                H_r = H_OBC_Kitaev_LR_QP(params, mu, length, AA=True)
                evals, evecs = np.linalg.eigh(H_r)
                evecs = evecs.T    

                if len(energies_r) == 0:
                    energies_r = evals
                else:
                    energies_r = np.vstack((energies_r, evals))

                vec_w[i] = Real_space_Kitaev_LR_QP_wn(params, mu, length, AA=True)
                
            energies_r = energies_r.T
            energies_array = np.append(energies_array, energies_r).reshape(len(energies_array)+1, len(energies_r), len(energies_r[0]))
            wn_array = np.append(wn_array, vec_w).reshape(len(wn_array)+1, len(vec_w))
            
        np.save(path_data + 'critical_delta/wn_array_critical_delta_mu_%.2f_constant_%s_it_%.0f.npy' %(mu, constant, it), wn_array)            
        np.save(path_data + 'critical_delta/energies_array_critical_delta_mu_%.2f_constant_%s_it_%.0f.npy' %(mu, constant, it), energies_array)

# CROSSINGS AT ZERO ENERGY

In [ ]:
it= 12
Fn1, Fn = Fibonacci(it)
length = 1

mu_list = np.arange(-1.5,1.5,0.05)
phase_list = [0, np.pi/2, np.pi, 3*np.pi/2]

for phase in phase_list:
    
    energies_r = overlap1 = overlap2 = []
    i = 0
    for mu in mu_list:

        params = dict(
            alpha = 0.5,
            delta = 1/2,
            t = -1,
            Fn1 = Fn1,
            Fn = Fn,
            phase = phase,
            constant= False
        )

        H_r = H_OBC_Majoranas_Kitaev_LR_QP(params, mu, length, AA=True)
        evals, evecs = np.linalg.eigh(H_r)
        evecs = evecs.T

        if len(energies_r) == 0:
            energies_r = evals
        else:
            energies_r = np.vstack((energies_r, evals))

        indices = np.argsort(abs(evals))
        evals_sorted = evals[indices]
        evecs_sorted = evecs[indices]
        es1 = evecs_sorted[0]
        eval1 = evals_sorted[0]
        es2 = evecs_sorted[1]
        eval2 = evals_sorted[1]

        if eval1 >= 0:
            edge_state1 = es1
            edge_state2 = es2
        else:
            edge_state1 = es2
            edge_state2 = es1        

        if i == 0:
            e1_array = edge_state1
            e2_array = edge_state2     
        else:
            e1_array = np.vstack((e1_array, edge_state1))
            e2_array = np.vstack((e2_array, edge_state2))
            
        i+=1
        
    energies_r = energies_r.T
    np.save(path_data + 'crossings/crossing_bands_phase_%.2f_it_%.0f.npy' %(phase,it), energies_r)

    for i in range(len(e1_array)-1):
        overlap1 = np.append(overlap1, abs(np.dot(np.conjugate(e1_array[i]),e1_array[i+1])))
        overlap2 = np.append(overlap2, abs(np.dot(np.conjugate(e2_array[i]),e2_array[i+1])))

    np.save(path_data + 'crossings/crossing_quantum_fidelity_1_phase_%.2f_it_%.0f.npy' %(phase,it), overlap1)
    np.save(path_data + 'crossings/crossing_quantum_fidelity_2_phase_%.2f_it_%.0f.npy' %(phase,it), overlap2)

## QUANTUM FIDELITY IN TERMS OF ALPHA AND MU

In [ ]:
it= 12
Fn1, Fn = Fibonacci(it)
length = 1

phase = np.pi
alpha_list = np.linspace(0.1,2,100)
mu_list = np.linspace(-1.5,0,100)
gaps = []
overlap1 = overlap2 = []

for alpha in alpha_list:
    
    i = 0
    overlap_mu_1 = overlap_mu_2 = []    
    gaps_mu = []
    
    for mu in mu_list:
    
        params = dict(
            alpha = alpha,
            delta = 1/2,
            t = -1,
            Fn1 = Fn1,
            Fn = Fn,
            phase = phase,
            constant= False
        )

        H_r = H_OBC_Majoranas_Kitaev_LR_QP(params, mu, length, AA=True)
        evals, evecs = np.linalg.eigh(H_r)
        evecs = evecs.T

        indices = np.argsort(abs(evals))
        evals_sorted = evals[indices]
        evecs_sorted = evecs[indices]
        es1 = evecs_sorted[0]
        eval1 = evals_sorted[0]
        es2 = evecs_sorted[1]
        eval2 = evals_sorted[1]

        gaps_mu = np.append(gaps_mu, abs(eval1-eval2))
        
        if eval1 >= 0:
            edge_state1 = es1
            edge_state2 = es2
        else:
            edge_state1 = es2
            edge_state2 = es1        

        if i == 0:
            e1_array = edge_state1
            e2_array = edge_state2     
        else:
            e1_array = np.vstack((e1_array, edge_state1))
            e2_array = np.vstack((e2_array, edge_state2))
            
        i+=1                    
        
    for j in range(len(e1_array)-1):
        overlap_mu_1 = np.append(overlap_mu_1, abs(np.dot(np.conjugate(e1_array[j]),e1_array[j+1])))
        overlap_mu_2 = np.append(overlap_mu_2, abs(np.dot(np.conjugate(e2_array[j]),e2_array[j+1])))

    if len(overlap1) == 0:
        overlap1 = overlap_mu_1
    else:
        overlap1 = np.vstack((overlap1, overlap_mu_1))

    if len(overlap2) == 0:
        overlap2 = overlap_mu_2
    else:
        overlap2 = np.vstack((overlap2, overlap_mu_2))
    if len(gaps) == 0:
        gaps = gaps_mu
    else:
        gaps = np.vstack((gaps, gaps_mu))
        
np.save(path_data + 'crossings_mu_alpha.npy', gaps)
np.save(path_data + 'overlap2_mu_alpha.npy', overlap2)
np.save(path_data + 'overlap1_mu_alpha.npy', overlap1)

## REAL-SPACE WINDING NUMBER IN TERMS OF ALPHA AND MU

In [ ]:
it = 12
length = 1
Fn1, Fn = Fibonacci(it)

mu_list = np.linspace(-1.5,1.5,100)
phase_list = np.linspace(0,2*np.pi,100)
wn_array = []

for phase in phase_list:
    
    vec_w = np.zeros_like(mu_list)
    i=0
    
    for mu in mu_list:

        params = dict(
            alpha = 0.5,
            delta = 0.5,
            t = -1,
            Fn1 = Fn1,
            Fn = Fn,
            phase = phase,
            constant= False
        )

        vec_w[i] = Real_space_Kitaev_LR_QP_wn(params, mu, length, AA=True)   
        i+=1
    wn_array = np.append(wn_array, vec_w).reshape(len(wn_array)+1, len(vec_w))

np.save(path_data + 'wn_phi_mu.npy', wn_array)

phase = np.pi
alpha_list = np.linspace(0.1,2,100)
mu_list = np.linspace(-1.5,0,100)
wn_array = []

for alpha in alpha_list:
    
    vec_w = np.zeros_like(mu_list)
    i=0
    
    for mu in mu_list:

        params = dict(
            alpha = alpha,
            delta = 0.5,
            t = -1,
            Fn1 = Fn1,
            Fn = Fn,
            phase = phase,
            constant= False
        )

        vec_w[i] = Real_space_Kitaev_LR_QP_wn(params, mu, length, AA=True)   
        i+=1
    wn_array = np.append(wn_array, vec_w).reshape(len(wn_array)+1, len(vec_w))

np.save(path_data + 'wn_mu_alpha.npy', wn_array)

## GAP BETWEEN MDMs IN TERMS OF ALPHA AND MU

In [ ]:
it = 12
length = 1
Fn1, Fn = Fibonacci(it)

phase = np.pi
alpha_list = np.linspace(0.1,2,100)
mu_list = np.linspace(-1.5,0,100)
gaps_array = []

for alpha in alpha_list:
    print('alpha', alpha)    
    vec_gaps = np.zeros_like(mu_list)
    i=0
    
    for mu in mu_list:

        params = dict(
            alpha = alpha,
            delta = 0.5,
            t = -1,
            Fn1 = Fn1,
            Fn = Fn,
            phase = phase,
            constant= False
        )

        H_r = H_OBC_Kitaev_LR_QP(params, mu, length, AA=True)
        evals, evecs = np.linalg.eigh(H_r)
        vec_gaps[i] = np.sort(abs(evals))[2]  
        i+=1
    gaps_array = np.append(gaps_array, vec_gaps).reshape(len(gaps_array)+1, len(vec_gaps))

np.save(path_data + 'gaps_mu_alpha.npy', gaps_array)

# SCALING OF THE EDGE STATES AT ALPHA 0.8,1.1

In [ ]:
it= 12
Fn1, Fn = Fibonacci(it)

length_list = [1,2,3,4,5,10,15,20,25]
alpha_list = [0.8,1.1]
mu_list = [0,-1]
 
for mu in mu_list:
    
    gaps_array = []
    
    for i, alpha in enumerate(alpha_list):
        
        gaps = []

        for L in length_list:

            params = dict(
                alpha = alpha,
                delta = 1/2,
                t = -1,
                Fn1 = Fn1,
                Fn = Fn,
                phase = np.pi,
                constant= False
            )

            H_r = H_OBC_Majoranas_Kitaev_LR_QP(params, mu, L ,AA=True)
            evals, evecs = np.linalg.eigh(H_r)

            gaps = np.append(gaps, 2*np.min(np.sort(abs(evals))))

        if i == 0:
            gaps_array = gaps
        else:
            gaps_array = np.vstack((gaps_array, gaps))

        np.save(path_data + 'scaling_gaps.npy' , gaps_array)

In [ ]:
it= 12
Fn1, Fn = Fibonacci(it)

D_list = [5,10,15,20,25,30,35,40,45,50]
alpha_list = [0.8,1.1]
mu_list = np.array([0,-1],dtype=float)
 
for mu in mu_list:
    
    wn_inf_array = []
    
    for i,alpha in enumerate(alpha_list):
        
        wn_inf = []

        for D in D_list:

            params = dict(
                alpha = alpha,
                delta = 1/2,
                t = -1,
                Fn1 = Fn1,
                Fn = Fn,
                phase = np.pi,
                constant= False
            )

            delta_k = 2*np.pi/D
            veck = np.arange(delta_k, 2*np.pi, delta_k)
            wn_inf = np.append(wn_inf, Chiral_Kitaev_QP_LR_wn(params, [mu], veck, AA=True))

        if i == 0:
            wn_inf_array = wn_inf
        else:
            wn_inf_array = np.vstack((wn_inf_array, wn_inf))
            
        np.save(path_data + 'scaling_wn.npy', wn_inf_array)

# AAH EDGE STATES

## CROSSINGS FOR SHORT RANGE AND LONG RANGE

In [3]:
it = 12
length = 1
Fn1, Fn = Fibonacci(it)

mu = 1

phase_list = np.linspace(0, 2*np.pi, 201)

for delta in [0,0.3]:
    
    if delta == 0.3:
        alpha_list = [0.5,5]
    else:
        alpha_list = [0]
        
    for alpha in  alpha_list:
        
        energies_r = []

        for phase in phase_list:

            params = dict(
                alpha = alpha,
                delta = delta,
                t = -1,
                Fn1 = Fn1,
                Fn = Fn,
                phase = phase,
                constant= False
            )

            H_r = H_OBC_Kitaev_LR_QP(params, mu, length, AA=True)
            evals, evecs = np.linalg.eigh(H_r)

            if len(energies_r) == 0:
                energies_r = evals
            else:
                energies_r = np.vstack((energies_r, evals))

        energies_r = energies_r.T
        np.save(path_data + 'AA_states/crossings_phase_delta_%.2f_alpha_%.2f_mu_%.2f.npy' %(delta,alpha,mu),energies_r)

## QUANTUM FIDELITY FOR SHORT RANGE AND FOR LONG RANGE

In [8]:
it = 12
length = 1
Fn1, Fn = Fibonacci(it)

mu = 1

phase_list = np.linspace(4.5,5.7,50)

for delta in [0, 0.3]:
    
    if delta == 0.3:
        alpha_list = [0.5,5]
        line_list = [0.85,0.85]
    else:
        alpha_list = [0]
        line_list = [0.9]
        
    for alpha, line in  zip(alpha_list, line_list):
        
        phase = phase_list[0]
        
        params = dict(
            alpha = alpha,
            delta = delta,
            t = -1,
            Fn1 = Fn1,
            Fn = Fn,
            phase = phase,
            constant= False
        )

        H_r = H_OBC_Kitaev_LR_QP(params, mu, length, AA=True)
        evals, evecs = np.linalg.eigh(H_r)
        evecs = evecs.T 

        evecs = evecs[(evals<line)]
        evals = evals[(evals<line)]
            
        eval1 = evals[-1]
        eval2 = evals[-2]
        index1 = np.arange(len(evals))[evals == eval1][0]
        index2 = np.arange(len(evals))[evals == eval2][0]
        
        evecs1 = evecs[index1]
        evecs2 = evecs[index2]  
        evals1 = evals[index1]
        evals2 = evals[index2]
                
        for phase in phase_list[1:]:

            params = dict(
                alpha = alpha,
                delta = delta,
                t = -1,
                Fn1 = Fn1,
                Fn = Fn,
                phase = phase,
                constant= False
            )

            H_r = H_OBC_Kitaev_LR_QP(params, mu, length, AA=True)
            evals, evecs = np.linalg.eigh(H_r)
            evecs = evecs.T 
            
            if len(evecs1) == 0:
                evecs1 = evecs[index1]
                evecs2 = evecs[index2]                
            else:
                evecs1 = np.vstack((evecs1, evecs[index1]))
                evecs2 = np.vstack((evecs2, evecs[index2]))
            evals1 = np.append(evals1, evals[index1])
            evals2 = np.append(evals2, evals[index2])       
            
        np.save(path_data + 'AA_states/crossing_bands_alpha_%.2f_delta_%.2f_mu_%.2f.npy' %(alpha,delta,mu), np.vstack((evals1,evals2)))

        overlap1 = overlap2 = []
        for i in range(len(evecs1)-1):
            overlap1 = np.append(overlap1, abs(np.dot(np.conjugate(evecs1[i]),evecs1[i+1])))
            overlap2 = np.append(overlap2, abs(np.dot(np.conjugate(evecs2[i]),evecs2[i+1])))
            
        np.save(path_data + 'AA_states/quantum_fidelity_alpha_%.2f_delta_%.2f_mu_%.2f.npy' %(alpha,delta,mu), np.vstack((overlap1,overlap2)))

## EDGE STATES

In [3]:
it = 12
length = 1
Fn1, Fn = Fibonacci(it)

mu = 1.2
phase = 5.07
alpha = 0.5

delta_list = np.linspace(0.1,0.5,20)

delta = 0.1
params = dict(
    alpha = alpha,
    delta = delta,
    t = -1,
    Fn1 = Fn1,
    Fn = Fn,
    phase = phase,
    constant= False
)

H_o = H_OBC_Kitaev_LR_QP(params, mu, length, AA =True)
evals_o, evecs_o = np.linalg.eigh(H_o)
evecs_o = evecs_o.T
 
edge_states = evals_o[(evals_o>0.4) & (evals_o<0.6)] 

eval1 = edge_states[0]
eval2 = edge_states[1]
index1 = np.arange(len(evals_o))[evals_o == eval1][0]
index2 = np.arange(len(evals_o))[evals_o == eval2][0]
    
evals1 = eval1
evals2 = eval2
evecs1 = evecs_o[index1]
evecs2 = evecs_o[index2]
energies_r = evals_o
    
for delta in delta_list[1:]:
    
    params['delta'] = delta
    
    H_o = H_OBC_Kitaev_LR_QP(params, mu, length, AA=True)
    evals_o, evecs_o = np.linalg.eigh(H_o)
    evecs_o = evecs_o.T
    
    energies_r = np.vstack((energies_r, evals_o))
    evals1 = np.append(evals1, evals_o[index1])
    evals2 = np.append(evals2, evals_o[index2])
    evecs1 = np.vstack((evecs1, evecs_o[index1]))
    evecs2 = np.vstack((evecs2, evecs_o[index2]))

energies_r = energies_r.T
np.save(path_data + 'AA_states/cdelta_bands_mu_%.2f_phase_%.2f_it_%.0f.npy' %(mu, phase,it), energies_r)
np.save(path_data + 'AA_states/cdelta_evals1_bands_mu_%.2f_phase_%.2f_it_%.0f.npy' %(mu, phase,it), evals1)
np.save(path_data + 'AA_states/cdelta_evals2_bands_mu_%.2f_phase_%.2f_it_%.0f.npy' %(mu, phase,it), evals2)
np.save(path_data + 'AA_states/cdelta_evecs1_bands_mu_%.2f_phase_%.2f_it_%.0f.npy' %(mu, phase,it), evecs1)
np.save(path_data + 'AA_states/cdelta_evecs2_bands_mu_%.2f_phase_%.2f_it_%.0f.npy' %(mu, phase,it), evecs2)

In [5]:
it = 12
length = 1
Fn1, Fn = Fibonacci(it)

mu = 1.20
phase = 5.07
delta = 0.3

alpha_list = np.linspace(0.1,3,30)

alpha = 0.1
params = dict(
    alpha = alpha,
    delta = delta,
    t = -1,
    Fn1 = Fn1,
    Fn = Fn,
    phase = phase,
    constant= False
)

H_o = H_OBC_Kitaev_LR_QP(params, mu, length, AA =True)
evals_o, evecs_o = np.linalg.eigh(H_o)
evecs_o = evecs_o.T
 
edge_states = evals_o[(evals_o>0.4) & (evals_o<1)] 

eval1 = edge_states[0]
eval2 = edge_states[1]
index1 = np.arange(len(evals_o))[evals_o == eval1][0]
index2 = np.arange(len(evals_o))[evals_o == eval2][0]
    
evals1 = eval1
evals2 = eval2
evecs1 = evecs_o[index1]
evecs2 = evecs_o[index2]
energies_r = evals_o
    
for alpha in alpha_list[1:]:
    
    params['alpha'] = alpha
    
    H_o = H_OBC_Kitaev_LR_QP(params, mu, length, AA=True)
    evals_o, evecs_o = np.linalg.eigh(H_o)
    evecs_o = evecs_o.T
    
    energies_r = np.vstack((energies_r, evals_o))
    evals1 = np.append(evals1, evals_o[index1])
    evals2 = np.append(evals2, evals_o[index2])
    evecs1 = np.vstack((evecs1, evecs_o[index1]))
    evecs2 = np.vstack((evecs2, evecs_o[index2]))

energies_r = energies_r.T
np.save(path_data + 'AA_states/alpha_bands_mu_%.2f_phase_%.2f_it_%.0f.npy' %(mu, phase,it), energies_r)
np.save(path_data + 'AA_states/alpha_evals1_bands_mu_%.2f_phase_%.2f_it_%.0f.npy' %(mu, phase,it), evals1)
np.save(path_data + 'AA_states/alpha_evals2_bands_mu_%.2f_phase_%.2f_it_%.0f.npy' %(mu, phase,it), evals2)
np.save(path_data + 'AA_states/alpha_evecs1_bands_mu_%.2f_phase_%.2f_it_%.0f.npy' %(mu, phase,it), evecs1)
np.save(path_data + 'AA_states/alpha_evecs2_bands_mu_%.2f_phase_%.2f_it_%.0f.npy' %(mu, phase,it), evecs2)

## ENERGY GAP BETWEEN EDGE STATES COLORMAP

In [6]:
it = 12
length = 1
Fn1, Fn = Fibonacci(it)

phase = 5.07

delta_list = np.linspace(0.1,1,50)[:-30]
alpha_list = np.linspace(0.1,3,50)
mu_list = [1.2]
#mu_list = np.arange(0.5,1,0.1) - evals>0.3

for mu in mu_list:
 
    gaps = []

    for alpha in alpha_list[:-1]:

        delta = 0.1

        params = dict(
            alpha = alpha,
            delta = delta,
            t = -1,
            Fn1 = Fn1,
            Fn = Fn,
            phase = phase,
            constant= False
        )

        H_o = H_OBC_Kitaev_LR_QP(params, mu, length, AA =True)
        evals_o, evecs_o = np.linalg.eigh(H_o)
        evecs_o = evecs_o.T

        edge_states = evals_o[(evals_o>0.4) & (evals_o<1)] 

        eval1 = edge_states[0]
        eval2 = edge_states[1]
        index1 = np.arange(len(evals_o))[evals_o == eval1][0]
        index2 = np.arange(len(evals_o))[evals_o == eval2][0]

        evals1 = eval1
        evals2 = eval2
        evecs1 = evecs_o[index1]
        evecs2 = evecs_o[index2]
        energies_r = evals_o

        gaps_delta = abs(eval1-eval2)

        for delta in delta_list[1:-1]:

            params['delta'] = delta

            H_o = H_OBC_Kitaev_LR_QP(params, mu, length, AA=True)
            evals_o, evecs_o = np.linalg.eigh(H_o)
            evecs_o = evecs_o.T

            eval1 = evals_o[index1]
            eval2 = evals_o[index2]
            evec1 = evecs_o[index1]
            evec2 = evecs_o[index2]

            evals1 = np.append(evals1, eval1)
            evals2 = np.append(evals2, eval2)

            energies_r = np.vstack((energies_r, evals_o))

            gaps_delta = np.append(gaps_delta, abs(eval1-eval2))

        if len(gaps) == 0:
            gaps = gaps_delta
        else:
            gaps = np.vstack((gaps, gaps_delta))     
    
    np.save(path_data + 'AA_states/gaps_alpha_delta_mu_%.2f_it_%.0f.npy' %(mu,it), gaps)   

## QUANTUM FIDELITY COLORMAP

In [7]:
it = 12
Fn1, Fn = Fibonacci(it)
length = 1

mu_list = [1.2]
delta_list = np.linspace(0.1,1,50)[:-30]
alpha_list = np.linspace(0.1,3,50)
phase_list = np.linspace(5.06,5.09,10)

for mu in mu_list:
    
    crossings = []
    crossings_int = []

    delta = 0.1
    alpha = 0.1
    phase = 5.07
    params = dict(
        alpha = alpha,
        delta = delta,
        t = -1,
        Fn1 = Fn1,
        Fn = Fn,
        phase = phase,
        constant= False
    )
    H_o = H_OBC_Kitaev_LR_QP(params, mu, length, AA =True)
    evals_o, evecs_o = np.linalg.eigh(H_o)
    evecs_o = evecs_o.T

    if mu < 1:
        edge_states = evals_o[(evals_o>0.3) & (evals_o<1)] 
    else:
        edge_states = evals_o[(evals_o>0.4) & (evals_o<1)] 

    eval1 = edge_states[0]
    eval2 = edge_states[1]
    index1 = np.arange(len(evals_o))[evals_o == eval1][0]
    index2 = np.arange(len(evals_o))[evals_o == eval2][0]

    for alpha in alpha_list[:-1]:

        params['alpha'] = alpha  
        crossing_delta = crossing_int_delta = []

        for delta in delta_list[:-1]:

            params['delta'] = delta
            evec1_phases = []
            eval1_phases = eval2_phases = []

            for phase in phase_list:

                params['phase'] = phase
                H_o = H_OBC_Kitaev_LR_QP(params, mu, length, AA =True)
                evals_o, evecs_o = np.linalg.eigh(H_o)
                evecs_o = evecs_o.T

                eval1 = evals_o[index1]
                eval2 = evals_o[index2]
                evec1 = evecs_o[index1]
                evec2 = evecs_o[index2]

                if len(evec1_phases) == 0:
                    evec1_phases = evec1
                else:
                    evec1_phases = np.vstack((evec1_phases, evec1))

                eval1_phases = np.append(eval1_phases, eval1) 
                eval2_phases = np.append(eval2_phases, eval2)

            overlap1_list = []
            for i in range(len(evec1_phases)-1):
                overlap1 = abs(np.dot(np.conjugate(evec1_phases[i]),evec1_phases[i+1]))
                overlap1_list = np.append(overlap1_list, overlap1)

            crossing = np.min(overlap1_list)
            crossing_delta = np.append(crossing_delta, crossing)

        if len(crossings) == 0:
            crossings = crossing_delta
        else:
            crossings = np.vstack((crossings, crossing_delta))

        np.save(path_data + 'AA_states/overlap_alpha_delta_mu_%.2f_it_%.0f.npy' %(mu,it), crossings)

# 2D CHERN NUMBERS (NOT USED FOR THE PLOTS)

## CHERN NUMBER ALPHA

In [ ]:
it = 12
length = 1
Fn1, Fn = Fibonacci(it)

mu = 1
delta = 0.3
phase = 5.08
alpha_list = np.linspace(0.01, 3, 20)

energy_gaps =  [0.6]

for energy_gap in energy_gaps:
    
    for D in [5,10,15,20,25,30]:

        phases_array = []
        chern_list = []
        
        for alpha in alpha_list:

            delta_k = 2*np.pi/D
            delta_phi = 2*np.pi/30
            kx_list = np.arange(0, 2*np.pi, delta_k)
            ky_list = np.arange(0, 2*np.pi, delta_phi)

            params = dict(
                alpha = alpha,
                delta = delta,
                t = -1,
                Fn1 = Fn1,
                Fn = Fn,
                phase = 2,
                constant = False
            )

            chern, phases = compute_Kitaev_AA_wn_TKNN_inf(params, mu, kx_list, ky_list, energy_gap, AA = True)
            chern_list = np.append(chern_list, chern)
            if len(phases_array) == 0:
                phases_array = phases
            else:
                phases_array = np.vstack((phases_array,phases))
            
        np.save(path_data + 'AA_states/TKNN_alpha_phase_AA_Kitaev_eg_%.2f_D_%.2f_mu_%.2f_it_%.2f.npy' %(energy_gap,D,mu,it), phases_array)
        np.save(path_data + 'AA_states/TKNN_alpha_cherns_AA_Kitaev_eg_%.2f_D_%.2f_mu_%.2f_it_%.2f.npy' %(energy_gap,D,mu,it), chern_list)

## CHERN NUMBER DELTA

In [ ]:
it = 12
length = 1
Fn1, Fn = Fibonacci(it)

mu = 1
alpha = 0.5
phase = 5.08
delta_list = np.linspace(0.01, 0.5, 20)

energy_gaps =  [1]

for energy_gap in energy_gaps:
    
    for D in [5,10,15,20,25,30]:

        phases_array = []
        chern_list = []
        
        for delta in delta_list:

            delta_k = 2*np.pi/D
            delta_phi = 2*np.pi/30
            kx_list = np.arange(0, 2*np.pi, delta_k)
            ky_list = np.arange(0, 2*np.pi, delta_phi)

            params = dict(
                alpha = alpha,
                delta = delta,
                t = -1,
                Fn1 = Fn1,
                Fn = Fn,
                phase = phase,
                constant = False
            )

            chern, phases = compute_Kitaev_AA_wn_TKNN_inf(params, mu, kx_list, ky_list, energy_gap, AA = True)
            chern_list = np.append(chern_list, chern)
            if len(phases_array) == 0:
                phases_array = phases
            else:
                phases_array = np.vstack((phases_array,phases))
            
        np.save(path_data + 'AA_states/TKNN_cdelta_phase_AA_Kitaev_eg_%.2f_D_%.2f_mu_%.2f_it_%.2f.npy' %(energy_gap,D,mu,it), phases_array)
        np.save(path_data + 'AA_states/TKNN_cdelta_cherns_AA_Kitaev_eg_%.2f_D_%.2f_mu_%.2f_it_%.2f.npy' %(energy_gap,D,mu,it), chern_list)

# APPENDIX: WEAK BULK EDGE CORRESPONDENCE

In [7]:
it= 12
Fn1, Fn = Fibonacci(it)
length = 1

delta_list = np.linspace(-1,1,50)
energies_r = vec_w = []
mu = 0.5

for delta in delta_list:

    params = dict(
        alpha = 0.5,
        delta = delta,
        t = -1,
        Fn1 = Fn1,
        Fn = Fn,
        phase = 0,
        constant= True
        ) 
    
    H_r = H_OBC_Kitaev_LR_QP(params, mu, length, AA=True)
    evals, evecs = np.linalg.eigh(H_r)
    evecs = evecs.T

    if len(energies_r) == 0:
        energies_r = evals
    else:
        energies_r = np.vstack((energies_r, evals))
        
    vec_w = np.append(vec_w, Real_space_Kitaev_LR_QP_wn(params, mu, length, AA=True))

energies_r = energies_r.T
np.save(path_data + 'weak_bec_delta_energies_it_%.0f_mu_%.2f.npy' %(it, mu), energies_r)
np.save(path_data + 'weak_bec_delta_wn_it_%.0f_mu_%.2f.npy' %(it, mu), vec_w)